# 로또번호 데이터 스크래핑


In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [2]:
# 로또회차 리스트
lot_no = soup.find(id='dwrNoList').text.split('\n')
lot_no = lot_no[1:-1]
print(lot_no[-1], lot_no[0])


1 1023


In [7]:
for i in lot_no[10::-1]:
    print(i)

1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023


In [9]:
# Chrome driver 실행
driver = webdriver.Chrome('/Users/newrandom/Desktop/파이썬 연습/challenge/lotto_project/Create_mode/chromedriver')
driver.get(url)

/var/folders/fd/5j5ym25j3gvbkwl_65fbg_2w0000gn/T/ipykernel_40307/2856298800.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/newrandom/Desktop/파이썬 연습/challenge/lotto_project/Create_mode/chromedriver')


In [54]:
# 로또 회차 조회
df = {}
for i in lot_no[::-1]:
                # [start:end:term]

    driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)
                            # 회차 리스트 선택하고 키 보내기
    driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)
    driver.implicitly_wait(5)   # 브라우저 5초 대기

    # ===== 로또번호 받아오기 ==== #
    # 1. 회차정보 : i
    # 2. 번호 받기
    df[i] = []
    for ball in range(1, 7):
        ball_num = '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(ball)

        df[i].append(int(driver.find_element(By.XPATH, ball_num).text))
        driver.implicitly_wait(3)
    # bo_num = '//*[@id="article"]/div[2]/div/div[2]/div/div[2]/p/span'

TypeError: unhashable type: 'slice'

In [109]:
# dataframe으로 저장하기
import pandas as pd
dataframe = pd.DataFrame(data=df)
# dataframe.to_csv('lotto_result.csv')
dataframe = dataframe.transpose()
dataframe.to_csv('lotto_result.csv')
dataframe.to_excel('lotto_result.xlsx')

In [110]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1022 entries, 1 to 1022
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       1022 non-null   int64
 1   1       1022 non-null   int64
 2   2       1022 non-null   int64
 3   3       1022 non-null   int64
 4   4       1022 non-null   int64
 5   5       1022 non-null   int64
dtypes: int64(6)
memory usage: 55.9+ KB


In [10]:
driver.close()

-------------------
# 최신회차 정보 가져오기 == python 파일로 만들 것 (완료)

In [49]:
# import module
import pandas as pd
from selenium import * 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

# read_csv
dataframe = pd.read_csv('../result/lotto_result.csv', index_col = 0)
latest_result = int(dataframe.iloc[-1].name)
print('저장되어 있는 최신 회차 : {}회'.format(latest_result))

# 로또 홈페이지 최신 회차 조회_1 : 홈페이지 접속
service = Service('../chromedriver')        # == 추가 == # webdriver 의 service 활용
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
driver = webdriver.Chrome(service=service)  # == 추가 == # 크롬드라이버 서비스 받아오기
driver.get(url)

# 로또 홈페이지 최신 회차 조회_2 : 최신회차조회
new = int(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').text.strip().split()[0])
print('최신 회차 : {}회'.format(new))

# 최신회차 검사
if new in dataframe.index:
    print('최신 회차 ({})가 저장되어 있습니다.'.format(new))

else:
    print('최신 회차 ({})가 저장되어 있지 않습니다. (현재 저장되어있는 최신회차 : {}회)'.format(new, latest_result))
    q = int(input('최신회차 정보를 업데이트 하시겠습니까? (1 : yes, 2: no)'))
    if q == 1:
        
        for i in range(latest_result+1, new+1):     # 저장된 최신회차+1 부터 신규 회차까지루프 > 데이터 저장
            
            driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)
            driver.implicitly_wait(10)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)

            nums = []
            for _ in range(1, 7):
                
                # 신규 회차의 로또번호 6개
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            print({i}, '회차가 저장됩니다.' , '{}'.format(nums))
            dataframe.loc[dataframe.iloc[-1].name + 1] = nums
        
        dataframe.to_csv('../result/lotto_result.csv')      # 최신 정보 저장

driver.close()

저장되어 있는 최신 회차 : 1022회
최신 회차 : 1024회
최신 회차 (1024)가 저장되어 있지 않습니다. (현재 저장되어있는 최신회차 : 1022회)
{1023} 회차가 저장됩니다. ['10', '14', '16', '18', '29', '35']
{1024} 회차가 저장됩니다. ['9', '18', '20', '22', '38', '44']


--------------

In [47]:
# dataframe.loc[1023] = [1,2,3,4,5,6]
dataframe.loc[dataframe.iloc[-1].name]

0     9
1    18
2    20
3    22
4    38
5    44
Name: 1024, dtype: object

In [39]:
dataframe.drop(1023, inplace=True)

In [42]:
dataframe

,0,1,2,3,4,5
1,12,27,29,38,41,45
2,1,3,20,25,36,45
3,6,7,13,15,21,43
4,14,27,30,31,40,42
5,5,12,17,29,34,35
...,...,...,...,...,...,...
1018,3,19,21,25,37,45
1019,1,4,13,17,34,39
1020,12,27,29,38,41,45
1021,12,15,17,24,29,45


In [48]:
driver.close()

In [23]:
if 1022 in dataframe.index:
    print('yes')
else:
    print('no')

yes


In [16]:
driver.close()

In [218]:
# import module
import pandas as pd
from selenium import * 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

# read_csv
dataframe = pd.read_csv('../result/lotto_result.csv', index_col = 0)
latest_result = int(dataframe.iloc[-1].name)
print('저장되어 있는 최신 회차 : {}회'.format(latest_result))

# 로또 홈페이지 최신 회차 조회_1 : 홈페이지 접속
service = Service('../chromedriver')        # == 추가 == # webdriver 의 service 활용
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
driver = webdriver.Chrome(service=service)  # == 추가 == # 크롬드라이버 서비스 받아오기
driver.get(url)

# 로또 홈페이지 최신 회차 조회_2 : 최신회차조회
new = int(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').text.strip().split()[0])
if new in dataframe.index:
    print('최신 회차 ({})가 저장되어있습니다.'.format(new))

# 로또 홈페이지 최신 회차 조회_3 : 최신회차 정보 업데이트 (미구현)
if new not in dataframe.index:
    print('최신 회차 ({})가 저장되어 있지 않습니다. (현재 저장되어있는 최신회차 : {}회)'.format(new, latest_result))
    q = int(input('최신회차 정보를 업데이트 하시겠습니까? (1 : yes, 2: no)'))
    if q == 1:
        a = new - latest_result     # 최신회차 - 저장된 최신회차 : new > latest_result
        # dataframe.loc[dataframe.index[-1]+1] = []
        for i in range(latest_result+1, new+1):     # 저장된 최신회차+1 부터 신규 회차까지루프 > 데이터 저장
            pass
        
        # < ============== 데이터를 저장하는 작동까지 넣을 것.


driver.close()

저장되어 있는 최신 회차 : 1022회


/var/folders/fd/5j5ym25j3gvbkwl_65fbg_2w0000gn/T/ipykernel_21906/820949709.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/newrandom/Desktop/파이썬 연습/challenge/chromedriver')


최신 회차 (1022)가 저장되어있습니다.


In [ ]:
driver.close()